# Tutorial for Remote Connection Features in MLCBase

[![PyPI](https://img.shields.io/pypi/v/mlcbase)](https://pypi.org/project/mlcbase/) &nbsp;
[![license](https://img.shields.io/github/license/wmchen/mlcbase.svg)](https://www.apache.org/licenses/LICENSE-2.0)

Author: [Weiming Chen](https://weimingchen.net) and [Yuanshuang Sun](https://www.mulingcloud.com/author/yuanshuang-sun/)

## Introduction

We support SSH and SFTP for remote connection.

Supported remote platform:
- Linux
- Windows

In [1]:
import sys
sys.path.append("../src")
from mlcbase import SSH, SFTP


👋 Welcome to use MuLingCloud. We aim to let everything easier.

📍 mlcbase (1.2.0.dev.202405) imported



In [2]:
host = ""      # change to the host of your remote sever
port = 22      # change to the port number of your remote sever
user = ""      # change to the username of your remote sever
password = ""  # change to the password of your remote sever

## 1. SSH

You can establish an SSH connection by instantiating `SSH()`.

### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `host` | str | The host of the remote server |
| `port` | int | The port number of the remote server |
| `user` | str | The user name used to login the remote server |
| `password` | str | The login password |
| `timeout` | int | The timeout for the connection in second. Defaults to 30 |
| `work_dir` | Optional[PathLikeType] | The working directory. If not None, will save the log file to "work_dir/log/". Defaults to None |
| `logger` | Optional[Logger] | Defaults to None |
| `quiet` | bool | Whether to set the logger as quiet mode. Defaults to False |

In [4]:
ssh_api = SSH(host, port, user, password)

2024-05-06 19:54:07 | 0 day(s) 00:00:00 | INFO | ssh connecting to remote server...
2024-05-06 19:54:08 | 0 day(s) 00:00:00 | SUCCESS | ssh connected to remote server.


You can send command to the remote server by calling `execute()`.

### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `command` | str | The command that you want to execute on the remote server |
| `return_str` | bool | Whether to return the `std_out` and `std_error` in `str`. Defaults to True |
| `encoding` | str | The encoding method. Defaults to "utf-8" |

### Return

It returns the `std_out` and `std_error` in `str` if `return_str` is True. Otherwise, it returns the `std_out` and `std_error` objects.

In [5]:
stdout, stderr = ssh_api.execute("mkdir testdir/")
if stdout:
    print(stdout)
if stderr:
    print(stderr)

In [6]:
stdout, stderr = ssh_api.execute("ls -a")
if stdout:
    print(stdout)
if stderr:
    print(stderr)

.
..
.bash_logout
.bashrc
.cache
examples.desktop
.profile
testdir



After using the ssh connection, don't forget to close the connection by calling `close()`.

In [7]:
ssh_api.close()

2024-05-06 19:54:08 | 0 day(s) 00:00:01 | INFO | ssh connection closed


## 2. SFTP

You can establish an SFTP connection by instantiating `SFTP()`.

### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `host` | str | The host of the remote server |
| `port` | int | The port number of the remote server |
| `user` | str | The user name used to login the remote server |
| `password` | str | The login password |
| `timeout` | int | The timeout for the connection in second. Defaults to 30 |
| `work_dir` | Optional[PathLikeType] | The working directory. If not None, will save the log file to "work_dir/log/". Defaults to None |
| `logger` | Optional[Logger] | Defaults to None |
| `quiet` | bool | Whether to set the logger as quiet mode. Defaults to False |

In [8]:
sftp_api = SFTP(host, port, user, password)

2024-05-06 19:54:08 | 0 day(s) 00:00:00 | INFO | sftp connecting to remote server...
2024-05-06 19:54:09 | 0 day(s) 00:00:00 | SUCCESS | sftp connected to remote server.


### 2.1 Upload a file

You can upload a file to the remote server by calling `upload_file()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `local_path` | PathLikeType | The local path of the file |
| `remote_path` | PathLikeType | The destination path on the remote server of the file |
| `remote_platform` | str | The type of the remote server. Current options including "linux" and "windows" |
| `callback` | Optional[Callable] | The callback function that used to visualize the progress of the transmission. Defaults to None |

#### Return

It returns True if success, otherwise returns False

In [9]:
sftp_api.upload_file(local_path="./examples/jsonfile.json",
                     remote_path="/home/testuser/testdir/jsonfile.json",
                     remote_platform="linux")

2024-05-06 19:54:09 | 0 day(s) 00:00:00 | INFO | uploading file: [LOCAL] ./examples/jsonfile.json -> [REMOTE] /home/testuser/testdir/jsonfile.json
2024-05-06 19:54:09 | 0 day(s) 00:00:00 | SUCCESS | file uploaded


True

You can define a callback function to visualize the progress of the transmission.

In [10]:
def show_progress(current, total):
    sys.stdout.write(f"Progress: {current}/{total} ({current/total*100:.2f}%)\r")
    sys.stdout.flush()
    if current == total:
        print(f"Progress: {current}/{total} ({current/total*100:.2f}%)")

In [11]:
sftp_api.upload_file(local_path="./examples/YOLOv9.pdf",
                     remote_path="/home/testuser/testdir/YOLOv9.pdf",
                     remote_platform="linux",
                     callback=show_progress)

2024-05-06 19:54:09 | 0 day(s) 00:00:00 | INFO | uploading file: [LOCAL] ./examples/YOLOv9.pdf -> [REMOTE] /home/testuser/testdir/YOLOv9.pdf


Progress: 4968643/4968643 (100.00%)


2024-05-06 19:54:13 | 0 day(s) 00:00:04 | SUCCESS | file uploaded


True

### 2.2 Download a file

You can download a file from the remote server by calling `download_file()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `remote_path` | PathLikeType | The remote path of the file |
| `local_path` | PathLikeType | The destination path on the local device of the file |
| `remote_platform` | str | The type of the remote server. Current options including "linux" and "windows" |
| `callback` | Optional[Callable] | The callback function that used to visualize the progress of the transmission. Defaults to None |

#### Return

It returns True if success, otherwise returns False

In [12]:
sftp_api.download_file(remote_path="/home/testuser/testdir/jsonfile.json",
                       local_path="./examples/jsonfile.remote_download.json",
                       remote_platform="linux")

2024-05-06 19:54:13 | 0 day(s) 00:00:04 | INFO | downloading file: [REMOTE] /home/testuser/testdir/jsonfile.json -> [LOCAL] ./examples/jsonfile.remote_download.json
2024-05-06 19:54:13 | 0 day(s) 00:00:04 | SUCCESS | file downloaded


True

### 2.3 Upload a directory

You can upload a directory to the remote server by calling `upload_dir()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `local_path` | PathLikeType | The local path of the directory |
| `remote_path` | PathLikeType | The destination path on the remote server of the directory |
| `remote_platform` | str | The type of the remote server. Current options including "linux" and "windows" |
| `callback` | Optional[Callable] | The callback function that used to visualize the progress of the process. Defaults to None |

#### Return

It returns True if success, otherwise returns False

In [13]:
sftp_api.upload_dir(local_path="./examples/example_dir",
                    remote_path="/home/testuser/testdir/example_dir",
                    remote_platform="linux")

2024-05-06 19:54:13 | 0 day(s) 00:00:05 | INFO | uploading directory: [LOCAL] ./examples/example_dir -> [REMOTE] /home/testuser/testdir/example_dir
2024-05-06 19:54:13 | 0 day(s) 00:00:05 | INFO | creating remote directory: /home/testuser/testdir/example_dir
2024-05-06 19:54:13 | 0 day(s) 00:00:05 | SUCCESS | directory created
2024-05-06 19:54:13 | 0 day(s) 00:00:05 | INFO | uploading file: [LOCAL] ./examples/example_dir\jsonfile.json -> [REMOTE] /home/testuser/testdir/example_dir/jsonfile.json
2024-05-06 19:54:13 | 0 day(s) 00:00:05 | SUCCESS | file uploaded
2024-05-06 19:54:13 | 0 day(s) 00:00:05 | INFO | uploading file: [LOCAL] ./examples/example_dir\xmlfile.xml -> [REMOTE] /home/testuser/testdir/example_dir/xmlfile.xml
2024-05-06 19:54:14 | 0 day(s) 00:00:05 | SUCCESS | file uploaded
2024-05-06 19:54:14 | 0 day(s) 00:00:05 | INFO | uploading file: [LOCAL] ./examples/example_dir\yamlfile.yaml -> [REMOTE] /home/testuser/testdir/example_dir/yamlfile.yaml
2024-05-06 19:54:14 | 0 day(s)

True

Using the a callback function to show the progress of the transmission is also available for uploading and downloading a directory.

### 2.4 Download a directory

You can download a directory from the remote server by calling `download_dir()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `remote_path` | PathLikeType | The remote path of the directory |
| `local_path` | PathLikeType | The destination path on the local device of the directory |
| `remote_platform` | str | The type of the remote server. Current options including "linux" and "windows" |
| `callback` | Optional[Callable] | The callback function that used to visualize the progress of the transmission. Defaults to None |

#### Return

It returns True if success, otherwise returns False

In [14]:
sftp_api.download_dir(remote_path="/home/testuser/testdir/example_dir",
                      local_path="./examples/example_dir.remote_download",
                      remote_platform="linux")

2024-05-06 19:54:14 | 0 day(s) 00:00:05 | INFO | downloading directory: [REMOTE] /home/testuser/testdir/example_dir -> [LOCAL] ./examples/example_dir.remote_download
2024-05-06 19:54:14 | 0 day(s) 00:00:05 | INFO | ./examples/example_dir.remote_download already exists, skip creating
2024-05-06 19:54:14 | 0 day(s) 00:00:06 | INFO | downloading file: [REMOTE] /home/testuser/testdir/example_dir/jsonfile.json -> [LOCAL] ./examples/example_dir.remote_download\jsonfile.json
2024-05-06 19:54:15 | 0 day(s) 00:00:06 | SUCCESS | file downloaded
2024-05-06 19:54:15 | 0 day(s) 00:00:06 | INFO | downloading file: [REMOTE] /home/testuser/testdir/example_dir/xmlfile.xml -> [LOCAL] ./examples/example_dir.remote_download\xmlfile.xml
2024-05-06 19:54:15 | 0 day(s) 00:00:06 | SUCCESS | file downloaded
2024-05-06 19:54:15 | 0 day(s) 00:00:06 | INFO | downloading file: [REMOTE] /home/testuser/testdir/example_dir/yamlfile.yaml -> [LOCAL] ./examples/example_dir.remote_download\yamlfile.yaml
2024-05-06 19:54:

True

Using the a callback function to show the progress of the transmission is also available for uploading and downloading a directory.

### 2.5 Other remote file operations

Supported methods:
- `remote_exists()`: check if a remote path exists
- `remote_is_file()`: check if a remote path is a file
- `remote_is_dir()`: check if a remote path is a directory
- `remote_mkdir()`: make a directory on the remote server
- `remote_listdir()`: list a remote directory

You can check if a remote path exists by calling `remote_exists()`.

In [15]:
sftp_api.remote_exists(remote_path="/home/testuser/testdir/jsonfile.json", remote_platform="linux")

True

You can check if a remote path is a file by calling `remote_is_file()` or if is a directory by calling `remote_is_dir()`.

In [16]:
sftp_api.remote_is_file(remote_path="/home/testuser/testdir/jsonfile.json", remote_platform="linux")

True

In [17]:
sftp_api.remote_is_dir(remote_path="/home/testuser/testdir/example_dir", remote_platform="linux")

True

You can make a remote directory by calling `remote_mkdir()`.

In [18]:
sftp_api.remote_mkdir(remote_path="/home/testuser/testdir/new_remote_dir", remote_platform="linux")

2024-05-06 19:54:16 | 0 day(s) 00:00:07 | INFO | creating remote directory: /home/testuser/testdir/new_remote_dir
2024-05-06 19:54:16 | 0 day(s) 00:00:07 | SUCCESS | directory created


True

You can list a remote directory by calling `remote_listdir()`.

In [19]:
sftp_api.remote_listdir(remote_path="/home/testuser/testdir", remote_platform="linux")

['/home/testuser/testdir/YOLOv9.pdf',
 '/home/testuser/testdir/new_remote_dir',
 '/home/testuser/testdir/jsonfile.json',
 '/home/testuser/testdir/example_dir']

### 2.6 Close connection

After using the sftp connection, don't forget to close the connection by calling `close()`.

In [20]:
sftp_api.close()

2024-05-06 19:54:16 | 0 day(s) 00:00:07 | INFO | sftp connection closed
